In [42]:

import numpy as np
import matplotlib.pyplot as plt
import os
import platform
import sys
import os 

sys.path.append("C:\\Program Files\\Lumerical\\v242\\api\\python") # this line is to direct the python script ot the lumerical python api 
import lumapi

current_directory= os.getcwd()
current_directory= r"{}".format(current_directory)
sys.path.append(current_directory)

focal_length = 100.0e-6
LensDiameter= 20.0e-6
SubstrateIndex= 1.46 
PillarIndex= 3.4757
period = 0.4e-6 
wavelength = 1.55e-6

unit_cell_library_file_name= 'EH_and_phase_vs_radius_interp_fdtd_si_sio2_sept13.mat'
zemax_phase_file_name=  'output_x_y_rad_apexPHAZE_optimized.txt'

In [91]:
fdtd= lumapi.FDTD()

setup_script= """
# create an aperture matching the diameter of the metalens 
R = getnamed("metalens","lens_radius");
W = getnamed("metalens::substrate","x span");
V = R*exp(1i*linspace(0,2*pi,200));
V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];
setnamed("aperture","vertices",V);
"""
fdtd.set("setup script",  setup_script)

radius_phase_file = current_directory+ '\\2_meta_library\\unit cell'+'\\' + unit_cell_library_file_name
zfile= current_directory+ '\\'+ zemax_phase_file_name

# #==================================================
# Add FDTD
# #==================================================
fdtd.addfdtd()
fdtd.set("dimension",2);  #  1 = 2D, 2 = 3D
fdtd.set("x",0)
fdtd.set("x span",25.0e-6)
fdtd.set("y",0)
fdtd.set("y span",25.0e-6)
fdtd.set("z",0.6e-6)
fdtd.set("z span",1.8e-6)
fdtd.set("mesh type", "auto non-uniform")
fdtd.set("mesh accuracy", 3)



# create an aperture matching the diameter of the metalens 
# myscript= """
# R = getnamed("metalens","lens radius");
# W = getnamed("metalens::substrate","x span");
# V = R*exp(1i*linspace(0,2*pi,200));
# V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];
# setnamed("aperture","vertices",V);
# """
# fdtd.set("script",myscript)

# #==================================================
# Add the metalens geometry 
# #==================================================
fdtd.addstructuregroup()
fdtd.set("name", "metalens")
fdtd.adduserprop("radius_vs_phase_data_file", 1, radius_phase_file)
fdtd.adduserprop("zemax_optimized_phase_file", 1,zfile)
fdtd.adduserprop("focal_length", 2, focal_length)
fdtd.adduserprop("lens_radius", 2, LensDiameter/2)
fdtd.adduserprop("substrate_index", 0, SubstrateIndex)
fdtd.adduserprop("pillar_index", 0, PillarIndex)



myscript =f"""deleteall;
matlabload(radius_vs_phase_data_file);
?mat_sub;
np = round({LensDiameter/2}/ {period});
x_mask = period * (-np:1:np);
y_mask = period * (-np:1:np);

addrect; # substrate
set("name", "substrate");
set("index",substrate_index);

# ## set("material", "sio2");
# if (mat_sub=="<Object defined dielectric>") {{
#     set("index",substrate_index);
# }}

set("x span",3*max(x_mask));
set("y span",3*max(x_mask));
set("z max",0);
set("z min",-2*{wavelength});

M=readdata(zemax_optimized_phase_file);
len= length(M)/3;
for(i=1:len) {{
    addcircle({{"name":"pillar", "x":M(i),"y":M(len+i),"radius":M(2*len+i)}});
}}
select("pillar");
set("index",index_pillar);
# set("material", "si");


set("z min",0);
set("z max",1.3e-6);
?height;
"""

fdtd.set("script", myscript)

# #==================================================
# Add the aperture
# #==================================================

fdtd.add2dpoly()
fdtd.set("name", "aperture")
fdtd.set("z", -0.05e-6)
R = LensDiameter/2
W = 3* 10e-6
apertureP= np.linspace(0, 2*np.pi, 200)
V = R*np.exp(complex(0,1)* apertureP)
#V = [W/2,0; W/2,-W/2; -W/2,-W/2; -W/2,W/2; W/2,W/2; W/2,0; real(V),imag(V)];
#fdtd.set("vertices", "[1.0e-6, 2.0e-6; 1.5e-6,2.0e-6]")
#  setnamed("aperture","vertices",V);

# #==================================================
# Add the source 
# #==================================================
fdtd.addplane()
fdtd.set("name", "source")
fdtd.set("injection axis","z")
fdtd.set("direction","forward")
fdtd.set("x",0)
fdtd.set("x span",30.0e-6)

fdtd.set("y",0)
fdtd.set("y span",30.0e-6)

fdtd.set("z",-0.15e-6)

fdtd.set("center wavelength", 1.55e-6)
fdtd.set("wavelength span",0.0e-6)

# #==================================================
# Add the FIELD monitor
# #==================================================
fdtd.addpower()
fdtd.set("name","field")
fdtd.set("monitor type",7)  # 2D z-normal
fdtd.set("x",0)
fdtd.set("x span",25.0e-6)
fdtd.set("y",0)
fdtd.set("y span",25.0e-6)
fdtd.set("z",1.365e-6)
# #==================================================

# #==================================================
# Add the Transmission monitor
# #==================================================
fdtd.addprofile()
fdtd.set("name","transmission")
fdtd.set("monitor type",7)  # 2D z-normal
fdtd.set("x",0)
fdtd.set("x span",25.0e-6)
fdtd.set("y",0)
fdtd.set("y span",25.0e-6)
fdtd.set("z",1.365e-6)
fdtd.set("output Pz", True)

fdtd.setglobalmonitor("frequency points", 1) # setting the global frequency resolution
fdtd.save("lumerical_full_lens_test")
fdtd.run()











# ## Step 4: Create the structures

# addrect; # substrate
# set("name","substrate");
# set("material", "sio2");
# if (mat_sub=="<Object defined dielectric>") {
#     set("index",index_sub);
# }
# set("x span",3*max(x_mask));
# set("y span",3*max(x_mask));
# set("z max",0);
# set("z min",-2*wavelength);


# M=readdata("..//output_x_y_rad_apexPHAZE_optimized.txt");
# len= length(M)/3;
# for(i=1:len) {
#     addcircle({"name":"pillar", "x":M(i),"y":M(len+i),"radius":M(2*len+i)});
    
# }


# select("pillar");
# set("index",index_pillar);
# set("material", "si");


# set("z min",0);
# set("z max",1.3e-6);
# ?height;
# set("detail",min([0,%pillar rendering detail%]));
# fdtd.adduserprop("radius",2,0.5e-6)

# myscript =      "addcircle; \n"
# myscript = myscript + "copy(1e-6); \n"
# myscript = myscript + "selectall; \n"
# myscript = myscript + "set(\"radius\",radius);"
# fdtd.set("name","dimer")
# fdtd.set("script",myscript)



In [ ]:
fdtd= lumapi.FDTD()
fdtd.load("lumerical_full_lens_test.fsp")
# #====================================================
monitor_name="transmission"
fdtd.getdata(monitor_name,"f")
T1=fdtd.transmission(monitor_name)
print(f"normalized power through surface near the metalens (using direct transmission): {T1}")


# second way to calculate power through a surface - transmission
x=fdtd.getdata(monitor_name,"x")
y=fdtd.getdata(monitor_name,"y")
f=fdtd.getdata(monitor_name,"f")
Pz=fdtd.getdata(monitor_name,"Pz")

print(Pz)
#Pz=fdtd.pinch(real(Pz))

# T2 = 0.5*integrate(Pz,1:2,x,y);  # power in watts
# T2 = T2/sourcepower(f);
# print("normalized power through surface near the metalens (by using Poynting vector): "+ num2str(T2));

# #--------------------------------------------------
# # calculate power same power using motor trans_1 and 
# # far field projection  
# #--------------------------------------------------
# # get focal length and the radius from the simulation
# R = getnamed("metalens","lens radius");
# TargetFocalLength = getnamed("metalens","focal length");

# # choose area to plot and spatial resolution
# x = linspace(-R,R,200);
# y = linspace(-R,R,200);
# z = linspace(1e-6,1.5*TargetFocalLength,100);




# # get the focal length 
# E2_z = farfieldexact3d('trans_1',0,0,z);
# E2_z = pinch(sum(abs(E2_z)^2,4));

# ind = find(E2_z,max(E2_z));
# zm = z(ind);
# SimulationFocalLength= zm;
# #---------------------------------------------------
# frequency= getresult("trans_1","f");
# wavelength= c/frequency;

# EFocal = farfieldexact3d('field', x, y, z= zm, {"field":"E"});

# Ex_Focal = pinch(EFocal, 4, 1);
# Ey_Focal = pinch(EFocal, 4, 2);
# Ez_Focal = pinch(EFocal, 4, 3);
# I_Focal = abs(Ex_Focal)^2 + abs(Ey_Focal)^2 ;


# #image(x*1e6,y*1e6,I_Focal,"x (mm)","y (mm)","Electric field at focal length "); 

# x_center= 0e-6;
# y_center= 0e-6;



# power_total = 0.5*sqrt(eps0/mu0)*integrate(I_Focal, 1:2, x, y);
# #print("total power at focal plane: "+ num2str(power_total/sourcepower(f)));
# #---------------------------------------------------
# ##------------------------------------------
# ## Find Full Width Half Maximum (FWHM) 
# ##------------------------------------------
# E2_x = farfieldexact3d('trans_1',x,0,z= zm);


# ##farfield in x diirection at the focal point (x=0,y=0,z=zm)
# Intensity_x = pinch(sum(abs(E2_x)^2,4));
# #plot(x*1e6,Intensity_x,'x (um)','|E|^2 (V^2/m^2)','Farfield at focal plane','linewidth=2');
# #legend("");

# p= fitnormpdf(x,Intensity_x);
# FWHM= 2 * sqrt(2* log(2))* p(3);
# #----------------------------
# FocalSpotRadius = FWHM*2 ;

# X = meshgridx(x,y);
# Y = meshgridy(x,y);
# #----------------------------
# filter = (FocalSpotRadius)>=sqrt((X-x_center)^2+(Y-y_center)^2);
# I_Focal_filtered =filter*pinch(I_Focal);
# #----------------------------
# Power_FocalSpot= 0.5*sqrt(eps0/mu0)*integrate(I_Focal_filtered, 1:2, x, y);
# Power_FocalSpot= Power_FocalSpot/sourcepower(f);
# #print("normalized power at focal spot: "+ num2str(Power_FocalSpot));
# #----------------------------
# FocusingEff = Power_FocalSpot/T1;
# #print("focusing efficiency in percent: "+ num2str(FocusingEff* 100));

# print("-------------------------");
# print("FWHM: "+ num2str(FWHM* 1e6));
# print("FOCAL: "+ num2str(zm* 1e6));
# print("EFF: "+ num2str(Power_FocalSpot* 100));
# print("TRANS: "+ num2str(T1* 100));
# print("FOCUSING EFF: "+ num2str(FocusingEff* 100));
# print("-------------------------");




In [102]:
Pz_real= np.real(Pz)
print(np.squeeze(Pz_real))

[[1.22106055e-06 1.23015304e-06 1.23984159e-06 ... 1.23983959e-06
  1.23015370e-06 1.22106189e-06]
 [1.25265409e-06 1.26189994e-06 1.27157876e-06 ... 1.27157925e-06
  1.26190075e-06 1.25265554e-06]
 [1.28556132e-06 1.29524256e-06 1.30517847e-06 ... 1.30517876e-06
  1.29524162e-06 1.28556126e-06]
 ...
 [1.28645652e-06 1.29656875e-06 1.30694294e-06 ... 1.30694164e-06
  1.29656877e-06 1.28645684e-06]
 [1.25400057e-06 1.26363571e-06 1.27371092e-06 ... 1.27371197e-06
  1.26363566e-06 1.25400098e-06]
 [1.22275308e-06 1.23219495e-06 1.24223718e-06 ... 1.24223731e-06
  1.23219500e-06 1.22275307e-06]]
